In [1]:
import pandas as pd              # v2.0.1
import numpy as np               # v1.24.3
import timeit
#import matplotlib.pyplot as plt  # v3.7.1
#import folium                    # v0.14.0
# adding a comment to test git integration

In [4]:
col_type = {'TaxYear' : 'int16', 'TotAppr' : 'int32', 'ParcelID' : 'str', 'Address' : 'str',
            'TotAssess' : 'int32', 'LandAppr' : 'int32', 'LivUnits' : 'int16', 'LandAcres' : 'float32',
            'LUCode' : 'category', 'TaxDist' : 'category'}
atlanta_df = pd.read_csv(r'CleanedAtlantaPropertyData_2009-2022.csv', sep=',', index_col=['ParcelID', 'TaxYear'],
                         dtype=col_type, low_memory=False)

In [71]:
atlanta_df.index.get_level_values(0).unique()

Index(['14 0245  LL2979', '14 0245  LL2987', '17 009800110426',
       '17 0225  LL0566', '17 015200071001', '14 0001  LL0217',
       '14 0001  LL0225', '14 0001  LL0399', '14 0001  LL0407',
       '14 0001  LL0563',
       ...
       '14 0042  LL2024', '14 0042  LL2032', '14 0042  LL2040',
       '14 0042  LL2057', '14 0009  LL2140', '14 0009  LL2298',
       '14 0009  LL2389', '14 0042  LL2115', '14 0042  LL2123',
       '17 0108  LL1764'],
      dtype='object', name='ParcelID', length=157953)

In [7]:
atlanta_df['TaxYear'].unique()
# There is no 2022 data. I don't know if they mislabeled the db or got 2023 done early.

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2023], dtype=int16)

In [50]:
atlanta_df.memory_usage(index=True, deep=False)

Index              132
OBJECTID      16549000
TaxYear        4137250
Owner         16549000
ParcelID      16549000
Address       16549000
AddrStreet    16549000
TaxDist        2069365
LUCode         4147378
NbrHood       16549000
TotAppr        8274500
TotAssess      8274500
LandAppr       8274500
LandAssess    16549000
LivUnits       4137250
LandAcres      8274500
dtype: int64

In [24]:
atlanta_df['Owner']

ParcelID         TaxYear
14 0245  LL2979  2009                                MALLORY TAMIKA
14 0245  LL2987  2009                                 BELL EDWINA S
17 009800110426  2009                          CB HABERSHAM III LLC
17 0225  LL0566  2009       HOUSING AUTH OF THE CITY OF ATLANTA THE
17 015200071001  2009                              CHAMPAGNE DEIDRA
                                             ...                   
17 010800011812  2023                             STEVERSON SUMMER 
17 010800011820  2023                             SHEHU SHEHU SULE 
17 010800011838  2023                                 PERRIE DAVID 
17 010800011846  2023                 ABRAHAMS AUBREY F & ANJOLI R 
17 010800011853  2023                          SAXTON RUSS CYNTHIA 
Name: Owner, Length: 2068625, dtype: object

In [22]:
# Doesn't work with multi-index
# for just Parcel / Owner questions 
atl_parcel_df = atlanta_df['ParcelID', 'TaxYear', 'Owner']
# remove leading / trailing whitespace and all punctuation.
atl_parcel_df['Owner'] = atl_parcel_df['Owner'].str.strip().str.replace(r'[^\w\s]',"", regex=True)

TypeError: Index(...) must be called with a collection of some kind, 'Owner' was passed

In [61]:
# Just test data so I'm not wrangling 2M rows all the time
t_df = pd.DataFrame({'TaxYear':[10,11,12,13,14,15,
                                10,11,12,13,14,15,
                                10,11,12,13,14,15],
                     'Owner':['Jim', 'Jim', 'Natalie', 'Jane', 'Jane', 'Natalie',
                             'Kim', 'Kim', 'James', 'Wendy', 'Wendy', 'Wendy',
                             'Tim', 'Tim', 'Tim','Tim', 'Tim', 'Tim'],
                     'ParcelID':[5,5,5,5,5,5,
                                 3,3,3,3,3,3,
                                 9,9,9,9,9,9]})
t_df.set_index(['ParcelID', 'TaxYear'], inplace=True)
t_df

Owner
ParcelID TaxYear         
5        10           Jim
         11           Jim
         12       Natalie
         13          Jane
         14          Jane
         15       Natalie
3        10           Kim
         11           Kim
         12         James
         13         Wendy
         14         Wendy
         15         Wendy
9        10           Tim
         11           Tim
         12           Tim
         13           Tim
         14           Tim
         15           Tim

In [26]:
p1 = ['14 0245  LL2979', '14 0245  LL2987', '17 009800110426']
temp = atlanta_df.loc[p1]
temp

OBJECTID                                      Owner  \
ParcelID        TaxYear                                                        
14 0245  LL2979 2009           10                             MALLORY TAMIKA   
                2010        36359                             MALLORY TAMIKA   
                2011       221332                            MALLORY TAMIKA    
                2012       255857                            MALLORY TAMIKA    
                2013       271799                            MALLORY TAMIKA    
                2014        81517                            MALLORY TAMIKA    
                2015       302751                            MALLORY TAMIKA    
                2016       271799                            MALLORY TAMIKA    
                2017       271799                            MALLORY TAMIKA    
                2018       140142                            MALLORY TAMIKA    
                2019       271799                            MALLORY TAMIKA    
                2020       271799                             BHADANI AKASH    
                2021       271799                             BHADANI AKASH    
                2023       271799                             BHADANI AKASH    
14 0245  LL2987 2009           11                              BELL EDWINA S   
                2010        36360                              BELL EDWINA S   
                2011       221333                             BELL EDWINA S    
                2012       255858                             BELL EDWINA S    
                2013       271798                             BELL EDWINA S    
                2014        81516                             BELL EDWINA S    
                2015       302750                             BELL EDWINA S    
                2016       271798                             BELL EDWINA S    
                2017       271798                             BELL EDWINA S    
                2018       140141                             BELL EDWINA S    
                2019       271798                             BELL EDWINA S    
                2020       271798                             BELL EDWINA S    
                2021       271798                             BELL EDWINA S    
                2023       271798                             BELL EDWINA S    
17 009800110426 2009           16                       CB HABERSHAM III LLC   
                2010        95519                              SBN TRUST THE   
                2011       278236                             SBN TRUST THE    
                2012       287495                             SBN TRUST THE    
                2013       121899                             SBN TRUST THE    
                2014       298572                             SBN TRUST THE    
                2015       176651  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2016       121899  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2017       121899  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2018        52798  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2019       121899  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2020       121899  MOSES ELIZABETH LINHART & MOSES MARILYN L   
                2021       121899                   MOSES ELIZABETH LINHART    
                2023       121899                   MOSES ELIZABETH LINHART    

                                         Address AddrStreet TaxDist LUCode  \
ParcelID        TaxYear                                                      
14 0245  LL2979 2009             904 SOCIETY CIR        NaN      05    107   
                2010             904 SOCIETY CIR        NaN      05    107   
                2011             904 SOCIETY CIR    SOCIETY      05    107   
                2012             904 SOCIETY CIR    SOCIETY      05    107   
                2013             904

In [110]:
test = atlanta_df[['TaxYear', 'Owner', 'ParcelID']].copy()#.set_index('ParcelID', drop=False)
# remove leading / trailing whitespace and all punctuation
test['Owner'] = test['Owner'].str.strip().str.replace(r'[^\w\s]',"", regex=True)
id = test.iloc[2]['ParcelID']
# need to find a scalable way to do this. Needs to be a copy to avoid SetWithCopy / bad practices.
test_sale = test[test['ParcelID']==id].copy()

#test_sale

In [111]:
# test[test['ParcelID']==test['ParcelID'][0]].set_index(['ParcelID', 'TaxYear'])
test.set_index(['ParcelID', 'TaxYear'])

,,Owner
ParcelID,TaxYear,
14 0245 LL2979,2009,MALLORY TAMIKA
14 0245 LL2987,2009,BELL EDWINA S
17 009800110426,2009,CB HABERSHAM III LLC
17 0225 LL0566,2009,HOUSING AUTH OF THE CITY OF ATLANTA THE
17 015200071001,2009,CHAMPAGNE DEIDRA
...,...,...
17 010800011812,2023,STEVERSON SUMMER
17 010800011820,2023,SHEHU SHEHU SULE
17 010800011838,2023,PERRIE DAVID


In [248]:
x3 = test_sale
x3['Transfer'] = x3['Owner'].ne(x3['Owner'].shift())
x3[['TaxYear', 'Transfer']]

,TaxYear,Transfer
2,2009,True
204749,2010,True
403527,2011,False
554605,2012,False
589927,2013,False
858659,2014,False
887378,2015,True
1034596,2016,False
1182405,2017,False
1344389,2018,False


In [66]:
# very very slow
def parcel_transfer(parcel_df, owner):
    '''takes all tax years for a parcel then iterates over the owner column and compares [owner][i] with initial
    owner. If the owner changes, the year is recorded and the new owner is used for subsequent comparisons'''
    parcel_sale = {parcel_df.TaxYear.iloc[0]: True}
    for i in range(parcel_df.shape[0]):
        if parcel_df.Owner.iloc[i] != owner:
            parcel_sale[parcel_df.TaxYear.iloc[i]] = True
            #parcel_sale[parcel_df.TaxYear.iloc[i]] = True
        owner = parcel_df.Owner.iloc[i]
        ''' This only works if parcel_df is it's own df. Views trigger SetWithCopy. IDK the best way to do this
        yet parcel_df['Transfer'] = parcel_df['TaxYear'].map(parcel_sale)'''
    #return(parcel_df)
    return parcel_sale

In [249]:
def fast_xfer(parcel_info):
    '''trying to improve on parcel_transfer function. Takes data for a single parcel in the form of
        [TaxYear], [ParcelID], and [Owner] then compares owner year to year.'''
    # only gives meaningful results if sorted by year ascending
    parcel = parcel_info.sort_values('TaxYear')
    parcel['Transfer'] = parcel['Owner'].ne(parcel['Owner'].shift()) # works but probably slow
    return parcel[['TaxYear', 'Transfer']]

In [256]:
def parcel_iterator(parcel_df):
    '''takes a database of ParcelID, Owner, and TaxYear then calls parcel_transfer for each parcelID'''
    parcels = parcel_df['ParcelID'].unique()
    tracker_dict = {}
    for parcel in parcels[:2]:
        tracker_dict[parcel] = fast_xfer(parcel_df[parcel_df['ParcelID']==parcel])
        #parcel_data = parcel_df[parcel_df['ParcelID']==parcel] # commented out for speed improvement? maybe? hopefully?
        #tracker_dict = parcel_transfer(parcel_data, parcel_data['Owner'].iloc[0])
    return tracker_dict #pd.DataFrame(tracker_dict).unstack().rename_axis(('ParcelID', 'index')).rename('Transfer')

In [144]:
# this takes 15 minutes? IDK how long
# started at  2305, no response at 2353
# still running at 07:17 finished at 18:39
# parcel_iterator(atl_parcel_df)

ParcelID         TaxYear
14 0245  LL2979  2009       True
                 2020       True
                 2010        NaN
                 2015        NaN
                 2021        NaN
                            ... 
17 0108  LL1764  2014        NaN
                 2016        NaN
                 2012        NaN
                 2019        NaN
                 2013        NaN
Name: Transfer, Length: 2211342, dtype: object

In [251]:
fast_xfer(test_sale)

,TaxYear,Transfer
2,2009,True
204749,2010,True
403527,2011,False
554605,2012,False
589927,2013,False
858659,2014,False
887378,2015,True
1034596,2016,False
1182405,2017,False
1344389,2018,False


In [275]:
parcel_iterator(test)

{'14 0245  LL2979':          TaxYear  Transfer
 0           2009      True
 173535      2010     False
 373620      2011     False
 538555      2012     False
 681969      2013     False
 773459      2014     False
 969957      2015     False
 1115815     2016     False
 1262880     2017     False
 1375452     2018     False
 1558610     2019     False
 1707107     2020      True
 1853638     2021     False
 2000024     2023     False,
 '14 0245  LL2987':          TaxYear  Transfer
 1           2009      True
 173536      2010     False
 373621      2011     False
 538556      2012     False
 681968      2013     False
 773458      2014     False
 969956      2015     False
 1115814     2016     False
 1262879     2017     False
 1375451     2018     False
 1558609     2019     False
 1707106     2020     False
 1853637     2021     False
 2000023     2023     False}

In [224]:
test_sale[test_sale['ParcelID'].isin(['14 0245  LL2979', '14 0245  LL2987'])]

Series([], Name: ParcelID, dtype: object)

### Transfer analysis
- [x]  create T/F column indicating if owner is new owner

In [67]:
xfers = parcel_transfer(test_sale, test_sale.Owner.iloc[0])
xfers

{2009: True, 2010: True, 2011: True, 2015: True, 2021: True}

In [30]:
xfers

{2009: True, 2010: True, 2011: True, 2015: True, 2021: True}

In [125]:
p1 = ['14 0245  LL2979', '14 0245  LL2979', '17 009800110426']
x3 = atlanta_df[atlanta_df['ParcelID'].isin(p1)].copy()


In [126]:
x4 = pd.merge(x3, parcel_iterator(test), on=['ParcelID', 'TaxYear'], how='outer')
x4.head()

,OBJECTID,TaxYear,Owner,ParcelID,Address,AddrStreet,TaxDist,LUCode,NbrHood,TotAppr,TotAssess,LandAppr,LandAssess,LivUnits,LandAcres,Transfer
0,10.0,2009,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,NaN,05,107,14593,0.0,0.0,0.0,NaN,1.0,NaN,True
1,16.0,2009,CB HABERSHAM III LLC,17 009800110426,3564 ROSWELL RD UNIT 55,NaN,05,107,1177,0.0,0.0,0.0,NaN,1.0,NaN,True
2,36359.0,2010,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,NaN,05,107,14593,86300.0,34520.0,8400.0,NaN,1.0,NaN,NaN
3,95519.0,2010,SBN TRUST THE,17 009800110426,3564 ROSWELL RD UNIT 55,NaN,05,107,1177,359000.0,143600.0,16720.0,NaN,1.0,NaN,True
4,221332.0,2011,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,47400.0,18960.0,12600.0,5040.0,1.0,0.019995,NaN


In [127]:
x4[x4['ParcelID']=='14 0245  LL2979']

,OBJECTID,TaxYear,Owner,ParcelID,Address,AddrStreet,TaxDist,LUCode,NbrHood,TotAppr,TotAssess,LandAppr,LandAssess,LivUnits,LandAcres,Transfer
0,10.0,2009,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,NaN,05,107,14593,0.0,0.0,0.0,NaN,1.0,NaN,True
2,36359.0,2010,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,NaN,05,107,14593,86300.0,34520.0,8400.0,NaN,1.0,NaN,NaN
4,221332.0,2011,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,47400.0,18960.0,12600.0,5040.0,1.0,0.019995,NaN
6,255857.0,2012,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,38400.0,15360.0,10200.0,4080.0,1.0,0.019995,NaN
9,271799.0,2013,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,34900.0,13960.0,10400.0,4160.0,1.0,0.019995,NaN
10,81517.0,2014,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,34900.0,13960.0,10400.0,4160.0,1.0,0.019995,NaN
13,302751.0,2015,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,91400.0,36560.0,8100.0,3240.0,1.0,0.019995,NaN
15,271799.0,2016,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,91400.0,36560.0,8100.0,3240.0,1.0,0.020000,NaN
17,271799.0,2017,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,81800.0,32720.0,8800.0,3520.0,1.0,0.020000,NaN
19,140142.0,2018,MALLORY TAMIKA,14 0245 LL2979,904 SOCIETY CIR,SOCIETY,05,107,14593,81800.0,32720.0,8800.0,3520.0,1.0,0.020000,NaN


## Data changes
- 2011 onward ['OWNER1'] --> ['Owner'] 
### Data Cleaning
- [ ] ['Owner']str.strip() out white space before analysis
## Analysis
- [x] List of parcels across all years  
    Parcel_iterator does the job fine but it would proably be less resource intensive to return a list of indicies where the owner changed.
    That would avoid most of the shenanigans of turning tracker_dict into a dataframe and transforming it into a more useful shape. I might try 
    to implement this later but for now everything works and I want to make progress.
- [x] which owners changed year to year
    - [ ]  How many ['Owner']s have more than one property
- [ ] How many parcels changed owners during a given year
- [ ] Parcels created / absorbed per year / ever
- [ ] property value year to year.
- [ ] Bar Chart sales v time
- [ ] Line graph property value
- [ ] Changes in livable units
- [ ] Local coverage of gentrification v tax records?  

| ParcelID | Owner | TaxDist | Subdiv | TotAssess | TotAppr | LivUnits |  
|----------|-------|---------|--------|-----------|---------|----------|  

- [ ] Whisker plot of value by tax district  
    - [ ] x value, y tax dist  
    - [ ] color code data  
        - Absolute value?  
        - Change per year?  
### Sales Analysis
- [x] Total number of parcels
    - [x] Change in parcel count over time
- [ ] Frequently sold parcels
    - [ ] Unique Owners
    - [ ] Year sold
- [ ] Most active TaxDist
- [ ] Highest TotAppr
- [ ] Sales per year  
### Parcel Sales
- [ ] Track parcel sales by year  
        - ['PARID'] Record year of sale  
        - Record if sale to LLC / INC / CO
- [ ] Most sold parcel  
- [ ] ['UNIT_NUM'] v sale frequency  
- [ ] Single family unit sales by year  
### parcel_sale algorithm
- [ ] Iterate over each ['ParcelID']
    - [ ] Check if ['Owner'] is the same  in each dataframe
    - [ ] add parcel info to new DF if ['Owner'] changes
    
    
#### This doesn't work how I wanted it to
- [x] Compare 2 data frames to see if the ['Owner'] stayed the same
    - [x] create a dataframe of [['ParcelID', 'AssessDifference']] 
    - [x] pd.concat(join='outer') sales + original dataframe
    - [x] move on to the next set of dataframes
    - [x] return appended dataframes


In [14]:
# find which years have data
years = atlanta_df['TaxYear'].unique()
# index on year. Could do drop=False but keeping the old df seems like a good idea currently.....(foreshadowing?)
year_df = atlanta_df.set_index('TaxYear')

In [15]:
# find properties that have changed owner in the past year.
prop = year_df[year_df.loc[2009]['Owner'].isin(year_df.loc[2010]['Owner']) == False]
prop

C:\Users\Owner\AppData\Local\Temp\ipykernel_23612\424640314.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prop = year_df[year_df.loc[2009]['Owner'].isin(year_df.loc[2010]['Owner']) == False]


KeyboardInterrupt: 

In [ ]:
# subset property owned by an LLC. Should expand to any company.
# LLC, LP, CORP, 
Parcels_10[Parcels_10['Owner'].str.contains('LLC')].head()

In [ ]:
# number of unique companies that own property. Should expand to any company.
# LLC, LP, CORP,
atlanta_df[atlanta_df['Owner'].str.contains('LLC')]['Owner'].unique().size

In [ ]:
# explore subdivision data
Parcels_15['SubdivBlck'].unique()

## WTF 
- Antioch Baptist Church Of Atlanta Cemetary  
    - Assessed to be worth $1.3B   
    - https://iaspublicaccess.fultoncountyga.gov/Datalets/Datalet.aspx?sIndex=0&idx=1  

## This is markdown ya goof!
year_list = [int(pd.unique(df['TaxYear'])) for df in dfs]  
parcel_data = pd.concat(dfs, keys=[year_list])
parcel_data

In [47]:
test = pd.DataFrame({'o': [1, 2, 3, 4, 5], 'p': [2, 3, 4, 5, 6], 'g': [3, 5, 3, 5, 2], 'id':[22, 22, 12, 18, 18]})
test

,o,p,g,id
0,1,2,3,22
1,2,3,5,22
2,3,4,3,12
3,4,5,5,18
4,5,6,2,18


In [54]:
index = [test['id'], test['g']]
idx = pd.MultiIndex.from_arrays(index)
idx

MultiIndex([(22, 3),
            (22, 5),
            (12, 3),
            (18, 5),
            (18, 2)],
           names=['id', 'g'])

In [65]:
t1 = test.set_index(idx, drop=True)
t1.drop(['g','id'],axis=1, inplace=True)
t1

o  p
id g      
22 3  1  2
   5  2  3
12 3  3  4
18 5  4  5
   2  5  6

In [83]:
t1.xs[3, axis=0, level=1]

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (153409513.py, line 1)

In [131]:
word = 'word!'

In [133]:
word.replace('.', ' ')

'word!'

## Old Code, don't use but maybe learn from? 

In [ ]:
# Only finds sales where the owner left atlanta the next tax year
def parcel_sale(df_list):
    ''' returns parcels sold where owner left Atlanta in a given year'''
    # feels like a kludgey way to select the next element
    i = 0
    leave_df = pd.DataFrame( {'ParcelID': [], 'TaxYear': []})
    for owner in df_list:
        n = i+1
        if i == len(df_list)-1:
            return sales_df
        else:
            sales = owner[owner['Owner'].isin(df_list[n]['Owner']) == False]
            leave_df = pd.concat([leave_df, sales[['ParcelID', 'TaxYear', 'TaxDist', 'TotAssess',
                                                   'TotAppr', 'LivUnits']]])
            i += 1     